# Подготовка датасетов

Мы собрали несколько датасетов на русском языке, направленных преимущественно на задачи STS, среди которых:
- https://huggingface.co/datasets/sberquad
- https://huggingface.co/datasets/RussianNLP/wikiomnia
- https://huggingface.co/datasets/RussianNLP/russian_super_glue
<!-- - https://github.com/vladislavneon/RuBQ/tree/master/RuBQ_2.0 -->



# Подготовка датасета для обучения

In [ ]:
# берем данные из датасетов, в которых все пары вопрос-ответ правильные
to_shafle = pd.concat([df_SQ,
                       df_muserc], ignore_index=True)
to_shafle.shape

Удаляем повторяющиеся контексты, чтобы при перемешивании не было правильных вариантов

Перемешиваем ответы на вопрос так, чтобы в соответствие каждому вопросу был поставлен неправильный(случайный) ответ

In [ ]:
import random

shuffled_context = to_shafle['context'].to_list()
random.shuffle(shuffled_context)
to_shafle['shuffled_context'] = shuffled_context

In [ ]:
to_shafle[to_shafle['context']==to_shafle['shuffled_context']]

In [ ]:
# Удалить повторяющиеся колонки при необходимости

to_shafle = to_shafle.drop([24165, 35062])

Всем строкам ставим label=0

In [ ]:
to_shafle['label'] = 0
to_shafle.drop(columns=['context'], inplace=True)
to_shafle.rename(columns={'shuffled_context': 'context'}, inplace=True)
to_shafle.head()

In [ ]:
to_shafle.shape

Объединяем датасет с правильными и неправильными парами вопрос-ответ

In [ ]:
train_data = pd.concat([train_data,
                        to_shafle], ignore_index=True)
train_data.shape

In [ ]:
train_data['label'].value_counts()

In [ ]:
# Сохраним датасет

train_data.to_csv('train_data.csv', index=False)

### Подготовка датасета для оценки


In [ ]:
dataset_vedomosti_articles = ("kartashoffv/vedomosti_articles") 

## Обучение cross-encoder PyTorch

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

from transformers import AutoTokenizer, AutoModel
from collections import OrderedDict



class Model(nn.Module):
    """
    labse - base bert-like model
    form labse i use pooler layer as intpu
    then classification head - binary classification to predict if this pair is TRUE question-answer
    """
    def __init__(self):
        super().__init__()
        # self.labse  = AutoModel.from_pretrained('./LaBSE-en-ru')
        # self.tokenizer = AutoTokenizer.from_pretrained('./LaBSE-en-ru', use_fast=True)
        # n_dim = 768
        # self.cls = nn.Sequential(OrderedDict(
        #                                       [
        #                                         ('dropout', torch.nn.Dropout(.0)),

        #                                         ('fc_1' , nn.Linear(n_dim, n_dim*2)),
        #                                         ('relu_1' , nn.GELU()),
        #                                         ('layernorm_1' , nn.LayerNorm(n_dim*2, eps=1e-12)),

        #                                         ('fc_2' , nn.Linear(n_dim*2, n_dim)),
        #                                         ('relu_2' , nn.GELU()),
        #                                         ('layernorm_2' , nn.LayerNorm(n_dim, eps=1e-12)),

        #                                         ('fc_3' , nn.Linear(n_dim, 2, bias=False)),
        #                                       ]
        #                         ))
        
    def forward(self, text):
        token = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
        model_output = self.labse(**token)
        result = self.cls(model_output.pooler_output)
        return result

In [ ]:
from collections import OrderedDict

d = OrderedDict({"a" : [1,2,3], "b" : 'abc'})

In [144]:
d.fromkeys('a')

OrderedDict([('a', None)])